# Notebook for OpenAI Pokemon Namen Generierung

Dieses Jupyter Notebook wurde entwickelt, um für jeden generierten synthetischen Datenpunkt einen passenden Pokémon-Namen zu generieren. Diese Generierung erfolgt mithilfe der OpenAI-Plattform und dient dazu, die Fähigkeiten von OpenAI im Bereich der kreativen Namensgebung zu demonstrieren.

#### Hintergrund

Die Generierung von Namen für synthetische Datenpunkte ist eine interessante Anwendung von KI-Technologien wie OpenAI. In diesem Notebook wird untersucht, wie gut OpenAI darin ist, Pokémon-Namen auf der Grundlage von gegebenen Statistiken zu erstellen. Dieser Ansatz ermöglicht es, automatisch und effizient eine große Anzahl von Namen zu generieren, die den vorgegebenen Kriterien entsprechen.

#### Ziel

Das Hauptziel dieses Notebooks besteht darin, die Qualität der von OpenAI generierten Pokémon-Namen zu bewerten. Hierbei wird auch ein Vergleich mit einem anderen LLM namens Llama 2 durchgeführt, um zu sehen, welches System bessere Namen erzeugt. 

## Imports

In [2]:
import os
import json
import pandas as pd

from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI

In [3]:
# load api_keys from json
with open('api_key.json') as json_datei:
    keys = json.load(json_datei)

In [4]:
os.environ['OPENAI_API_KEY'] = keys["open_ai_api_key"] # set api key

## Code

### llama-index OpenAI

In [5]:
# Dieses Promptemaple definiert die Vorlage für die Generierung eines Pokémon-Namens auf der Grundlage der angegebenen Werte. (Sehr wichtig um Halluzination zu vermeiden!)
# Stats include: Name, Type 1, Type 2, Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, Generation, Legendary, Evolution
# Examples:
# Input: Grass, Poison, 318, 45, 49, 49, 65, 65, 45, 1, False, 0; Output: Bulbasaur
# Input: Fire, NaN, 309, 39, 52, 43, 60, 50, 65, 1, False, 1; Output: Charmander
# Input: Fire, Water, 600, 80, 110, 120, 130, 90, 70, 6, True, 0; Output: Volcanion
# Only the name is output.
prompt_template= '<SYS> Generiere mir auf Basis dieser Stats einen Pokemon namen den es nicht gibt: Stats: Name, Type 1, Type 2, Total, HP, Attack, Defense, 	Sp. Atk,	Sp. Def,	Speed,	Generation,	Legendary, Evolution // Beispiele: Input:  Grass, Poison, 318, 45, 49,	49,	65,	65,	45,	1,	False, 0; Output: Bulbasaur Input: Fire, NaN,	309,	39,	52,	43,	60,	50,	65	1 False, 1; Output: Charmander Input: Fire, Water,	600,	80,	110,	120,	130,	90,	70,	6,	True, 0; Output: Volcanion	 Gib mir nur den Namen aus!'

In [6]:
def open_ai_pokemon_assistant(pokemon_stats):
    """ OpenAI Pokemon Assistant um Pokemon Namen anhand von einem String mit den Stats zu bewerten"""
    messages = [
        ChatMessage(role="system", content=prompt_template), # Systemnachricht mit dem vordefinierten prompt_template
        ChatMessage(role="user", content=pokemon_stats), # Pokemon_stats
    ]
    resp = OpenAI().chat(messages)
    return resp.message.content

In [11]:
#Datenvorbereinigung
syn_data = pd.read_csv("Gaussian_Pokemon.csv")
del syn_data["Unnamed: 0"]
syn_data = syn_data[0:10]
del syn_data["Name"]

# Wichtig um einen String zu generieren für das OpenAI Modul
syn_data['Attribut'] = syn_data['Type 1'] + ', ' + syn_data['Type 2'].astype(str) + ', ' + syn_data['HP'].astype(str) + ', ' + \
                        syn_data['Attack'].astype(str) + ', ' + syn_data['Defense'].astype(str) + ', ' + syn_data['Sp. Atk'].astype(str) + \
                        ', ' + syn_data['Sp. Def'].astype(str) + ', ' + syn_data['Speed'].astype(str) + ', ' + \
                        syn_data['Generation'].astype(str) + ', ' + syn_data['Legendary'].astype(str) + ', ' + \
                        syn_data['Evolution'].astype(str)

In [13]:
syn_data['Name_open_ai'] = syn_data["Attribut"].apply(open_ai_pokemon_assistant) # Assistant wird für jeden Datenpunkt (Pokemon) angewendet.
columns = ['Name_open_ai'] + [col for col in syn_data.columns if col != 'Name_open_ai'] # Name sollte ganz vorne stehen :)
syn_data = syn_data[columns]

In [17]:
syn_data.head(5)

,Name_open_ai,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Evolution,Attribut
0,Dragound,sdv-pii-64jr8,Ground,Dragon,81,93,99,88,83,71,6,False,0.0,"Ground, Dragon, 81, 93, 99, 88, 83, 71, 6, Fal..."
1,Aquafin,sdv-pii-xnuld,Water,NaN,32,24,29,43,34,39,1,False,1.0,"Water, nan, 32, 24, 29, 43, 34, 39, 1, False, 1.0"
2,Darklurk,sdv-pii-23pgo,Dark,NaN,45,28,22,35,27,71,5,False,1.0,"Dark, nan, 45, 28, 22, 35, 27, 71, 5, False, 1.0"
3,Flametrix,sdv-pii-8mult,Fire,NaN,80,101,80,135,129,64,1,False,2.0,"Fire, nan, 80, 101, 80, 135, 129, 64, 1, False..."
4,Blitzstrike,sdv-pii-pjbi7,Fighting,NaN,57,74,43,71,71,79,3,False,1.0,"Fighting, nan, 57, 74, 43, 71, 71, 79, 3, Fals..."


## LLama

In [ ]:
https://github.com/meta-llama/llama/blob/main/example_text_completion.py

In [18]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto"
)

base_prompt = "<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt}[/INST]"

def get_sentiment_llama(text):
    input = base_prompt.format(system_prompt = "Analyze the text in the content and evaluate the overall sentiment. Answer with just \"Positive\", \"Negative\", or \"Neutral\"",
                               user_prompt = text)
    print(input)
    
    sequences = pipeline(
        input,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
        return_full_text=False,
        temperature=0.5
    )

    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

get_sentiment_llama("Ok, first assesment of the kindle2 ...it fucking rocks")

/Users/marceldielacher/anaconda3/envs/GAI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-hf.
401 Client Error. (Request ID: Root=1-6627f1b5-344f9ad45755c3ad171203e9;2e394ef0-7ba1-49e6-a017-9c2aaf0b35be)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must be authenticated to access it.